In [1]:
!pip install -q ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/

In [2]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00


In [3]:
!pip install -q datasets

In [4]:
# if you use colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# If you use colab, this is colab_path, please change it to your own path to the code folder of this project
# WARNING: IF YOU ARE USING COLAB, THIS COULD ONLY BE RUN ONCE! OTHERWISE NOTEBOOK WILL CRASH
%cd /content/drive/MyDrive/chuangjl/code

/content/drive/MyDrive/chuangjl/code


In [6]:
# Miscellany
from tqdm.notebook import tqdm
import pandas as pd
import torch
from typing import Optional, List, Tuple
import os
import matplotlib.pyplot as plt

import locale
from ragatouille import RAGPretrainedModel

# langchain
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.text_splitter import RecursiveCharacterTextSplitter

# datasets
import datasets
from datasets import concatenate_datasets
from datasets import Dataset

# transformers
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Pipeline
from transformers import pipeline

In [7]:
def Metric(output, ref):
  ref = ref.lower().split()
  macGen = output.lower().split()
  setRef = set(ref)
  setGen = set(macGen)
  precision = 0
  setRefNew = set([])
  setGenNew = set([])
  for ele in setRef:
    x = ele.replace(",", "")
    x = x.replace(".", "")
    x = x.replace("?", "")
    x = x.replace("!", "")
    x = x.replace("\"", "")
    x = x.replace("'", "")
    x = x.replace(" ", "")
    x = x.replace("(", "")
    x = x.replace(")", "")
    setRefNew.add(x.strip())

  for ele in setGen:
    x = ele.replace(",", "")
    x = x.replace(".", "")
    x = x.replace("?", "")
    x = x.replace("!", "")
    x = x.replace("\"", "")
    x = x.replace("'", "")
    x = x.replace(" ", "")
    x = x.replace("(", "")
    x = x.replace(")", "")
    setGenNew.add(x.strip())

  setGen = setGenNew
  setRef = setRefNew
  # print(setGen, setRef)

  if len(setGen) != 0:
    precision = len(setRef.intersection(setGen)) / len(setGen)
  recall = 0
  if len(setRef) != 0:
    recall = len(setRef.intersection(setGen)) / len(setRef)
  f1 = 0
  if precision + recall != 0:
    f1 = 2 * (precision * recall) / (precision + recall)
  return recall, precision, f1

In [8]:
def recursive_split_documents_by_characters(
    chunk_size,
    chunk_overlap,
    raw_documents,
    tokenizer):
    # text splitter
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=chunk_size,
        chunk_overlap = chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
    )

    # split documents
    processed_documents = []
    for doc in raw_documents:
        processed_documents += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    unique_processed_documents = []
    for doc in processed_documents:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            unique_processed_documents.append(doc)

    return unique_processed_documents

In [9]:
data_base_path = "../database"

# Academics
handbooks_path = "Academics_LTI/Handbook"
program_path = "Academics_LTI/Programs_and_staff"
# Course
schedule_fall23_path = "Courses_CMU/Schedule/augmentFall23.txt"
schedule_spring24_path = "Courses_CMU/Schedule/augmentSpring24.txt"
schedule_summer24_path = "Courses_CMU/Schedule/augmentSummer24.txt"

calendar_2324_path = "Courses_CMU/Calendars/Calendar_scentences_2324.txt"
calendar_2425_path = "Courses_CMU/Calendars/Calendar_scentences_2425.txt"
# Events
commencement_path = "Events_CMU/Commencement/Commencement.txt"
spring_path = "Events_CMU/Spring/carnival_and_reunion.txt"
# Faculty
faculty_path = "Faculty_LTI/Faculty/LTI_faculty.txt"
researches_path = "Faculty_LTI/Research/"
# History
History_path = "History"

In [10]:
# system1
embed1 = "WhereIsAI/UAE-Large-V1"
embed2 = "thenlper/gte-large"
READER_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

num_of_trial = 5
percentage_of_documents = 0.3
num_of_questions = 5

In [11]:
embedding_tokenizer_1 =  AutoTokenizer.from_pretrained(embed1)
embedding_model_1 = HuggingFaceEmbeddings(
    model_name = embed1,
    multi_process = True,
    model_kwargs = {"device": "cuda"},
    encode_kwargs = {"normalize_embeddings": True},  # set True for cosine similarity
)

embedding_tokenizer_2 =  AutoTokenizer.from_pretrained(embed2)
embedding_model_2 = HuggingFaceEmbeddings(
    model_name = embed2,
    multi_process = True,
    model_kwargs = {"device": "cuda"},
    encode_kwargs = {"normalize_embeddings": True},  # set True for cosine similarity
)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization by replacing the Linear layers with FP4/NF4 layers from bitsandbytes.
    bnb_4bit_quant_type="nf4", # This sets the quantization data type in the bnb.nn.Linear4Bit layers. Options are FP4 and NF4 data types which are specified by fp4 or nf4.
    bnb_4bit_use_double_quant=True, # False # nested quantization where the quantization constants from the first quantization are quantized again.
    bnb_4bit_compute_dtype=torch.bfloat16, # This sets the computational type which might be different than the input time. For example, inputs might be fp32, but computation can be set to bf16 for speedups.
)

reader_tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
reader_model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [12]:
reader_pipeline = pipeline(
    model = reader_model,
    tokenizer = reader_tokenizer,
    task = "text-generation",
    do_sample = True,
    temperature = 0.1,
    repetition_penalty = 1.1,
    return_full_text = False,
    max_new_tokens = 700,
  )

In [13]:
import random
import locale
# subset of questions
questions = []
answers = []
with open("../data/test/TotalQ.txt", "r", encoding="utf-8") as f:
    for line in f:
        questions.append(line)
with open("../data/test/TotalA.txt", "r", encoding="utf-8") as f:
    for line in f:
        answers.append(line)

indexes = random.sample(range(len(questions)), num_of_questions)

questions = [questions[i] for i in indexes]
answers = [answers[i] for i in indexes]

In [14]:
print(len(questions))

5


In [15]:
few_shot_prompt = """Question: Is summer internship between the first and second year necessary for students in MSAII programs?
Answer: Yes
Question: In general, how many units of graduate-level courses must be passed for Ph.D. in Language and Information Technologies?
Answer: 96
Question: What is the date of 2025 Summer One & Mini-5 voucher deadline?
Answer: June 18, Wednesday in Summer 2025
Question: Who is the instructor of 11711 in Spring 2024
Answer:  Neubig
Question: When is Commencement Ceremony?
Answer: 10–11:30 a.m, Sunday, May 12
Question: What time of the day is Kiltie Band Concert?
Answer: 3:30 PM-4:30 PM ET
Question: What is Daphne Ippolito's Email?
Answer: daphnei@cmu.edu
Question:  Who is the author of the paper 'StyleRF: Zero-Shot 3D Style Transfer of Neural Radiance Fields'?
Answer: Eric P. Xing
Question: Who in 1986 wrote a white paper proposing the creation of "a School of Computer Science?
Answer: Nico Habermann and then-CMU provost Angel Jordan
Question: In what month do students and alumni reviewed potential mascot images in focus groups?
Answer: October"""

sys_prompt = [
    {"role": "user", "content": """Below are some documents as context {context}.""" },
    {"role": "assistant", "content": "Understood" },
    {"role": "user", "content": """Below are some example questions and their answers as reference {few_shot_prompt}.""" },
    {"role": "assistant", "content": "Understood" },
    {"role": "user", "content": "Based on these context documents, could you answer the following question? Question: {question}. Your answer should have the same format as the example answers. Your answer should not have any prefix or suffix. Your answer should not have any other information, including explanation, process, note, and so on. Do not give any other information other than the answer!"},
]

prompt_template = reader_tokenizer.apply_chat_template(
    sys_prompt, tokenize=False, add_generation_prompt=True
)
print(prompt_template)

def rag(
    question, llm, database,
    rerank_model = None,
    num_retrieved_docs = 40,
    num_return_docs = 5,
    few_shot_prompt = few_shot_prompt,
    print_output = True):
    # Gather documents with retriever
    if print_output: print("retrieving documents")
    rel_docs = database.similarity_search(query = question, k = num_retrieved_docs)
    rel_docs = [doc.page_content for doc in rel_docs]  # keep only the text

    # rerank results
    if rerank_model:
        if print_output: print("reranking")
        rel_docs = rerank_model.rerank(question, rel_docs, k = num_return_docs)
        rel_docs = [doc["content"] for doc in rel_docs]

    # select top k
    rel_docs = rel_docs[:num_return_docs]

    # build the final prompt
    context = "".join([f"Document {str(i)}: " + doc + "\n" for i, doc in enumerate(rel_docs)])
    final_prompt = prompt_template.format(few_shot_prompt = few_shot_prompt,
                                          context = context,
                                          question = question)

    # generating an answer
    if print_output: print("generating answer")
    answer = llm(final_prompt, pad_token_id=llm.tokenizer.eos_token_id)[0]["generated_text"]

    return final_prompt, answer

<s>[INST] Below are some documents as context {context}. [/INST]Understood</s>[INST] Below are some example questions and their answers as reference {few_shot_prompt}. [/INST]Understood</s>[INST] Based on these context documents, could you answer the following question? Question: {question}. Your answer should have the same format as the example answers. Your answer should not have any prefix or suffix. Your answer should not have any other information, including explanation, process, note, and so on. Do not give any other information other than the answer! [/INST]


In [22]:
# build total ds
ds = datasets.load_dataset("ge0rgeli/CMU_LTI")
ds = ds['train']
ds = ds.shuffle()

In [17]:
ps_1 = []
rs_1 = []
f1_1 = []

ps_2 = []
rs_2 = []
f1_2 = []

In [18]:
locale.getpreferredencoding = lambda: "UTF-8"
rerank_model = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [33]:
num_of_trial = 100
percentage_of_documents = 0.7
num_of_questions = 70

for i in range(num_of_trial):
  print(f"trial {i}")
  # 1. build sample documents
  raw_documents = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]
  raw_documents =  random.sample(raw_documents, int(percentage_of_documents * (len(raw_documents))))
  print(len(raw_documents))
  #
  # processed_documents_1
  processed_documents_1 = recursive_split_documents_by_characters(
      chunk_size = 512,  # We choose a chunk size adapted to our model
      chunk_overlap = 50,
      raw_documents = raw_documents,
      tokenizer = embedding_tokenizer_1,
  )
  # processed_documents_2
  processed_documents_2 = recursive_split_documents_by_characters(
      chunk_size = 512,  # We choose a chunk size adapted to our model
      chunk_overlap = 50,
      raw_documents = raw_documents,
      tokenizer = embedding_tokenizer_2,
  )
  # 2. build data base
  database_1 = FAISS.from_documents(processed_documents_1,
                                embedding_model_1,
                                distance_strategy=DistanceStrategy.COSINE)
  print("db1")
  database_2 = FAISS.from_documents(processed_documents_2,
                                embedding_model_2,
                                distance_strategy=DistanceStrategy.COSINE)
  print("db2")
  # 3. run the pipeline

  # 4. evaluate answer
  answer_1_list = []
  answer_2_list = []
  embed_1_name = embed1.split(sep = "/")[-1]
  embed_2_name = embed2.split(sep = "/")[-1]
  model_name = READER_MODEL_NAME.split(sep = "/")[-1]
  # -1 means all questions
  # set the number of questions you want to set
  question_count = 0
  for line in questions:
      if question_count == num_of_questions:
          break
      question = line
      final_prompt, answer_1 = rag(question = question,
                                llm = reader_pipeline,
                                database = database_1,
                                rerank_model = rerank_model,
                                print_output = False)
      final_prompt, answer_2 = rag(question = question,
                                llm = reader_pipeline,
                                database = database_2,
                                rerank_model = rerank_model,
                                print_output = False)
      print(question)
      answer_1 = answer_1.replace("\n", "")
      answer_1 = answer_1.strip()
      answer_2 = answer_2.replace("\n", "")
      answer_2 = answer_2.strip()
      print(answer_1)
      print(answer_2)
      answer_1_list.append(answer_1)
      answer_2_list.append(answer_2)
      question_count += 1
  # clean file
  # 5.
  psSum_1 = 0
  rsSum_1 = 0
  f1Sum_1 = 0
  psSum_2 = 0
  rsSum_2 = 0
  f1Sum_2 = 0
  i1 = 0
  i2 = 0
  for line1, line2 in zip(answer_1_list, answers):
    r, p, f = Metric(line1, line2)
    rsSum_1 += r
    psSum_1 += p
    f1Sum_1 += f
    i1 += 1
  for line1, line2 in zip(answer_2_list, answers):
    r, p, f = Metric(line1, line2)
    rsSum_2 += r
    psSum_2 += p
    f1Sum_2 += f
    i2 += 1
  ps_1.append(psSum_1/(i1))
  rs_1.append(rsSum_1/(i1))
  f1_1.append(f1Sum_1/(i1))

  ps_2.append(psSum_2/(i2))
  rs_2.append(rsSum_2/(i2))
  f1_2.append(f1Sum_2/(i2))

trial 0


  0%|          | 0/12937 [00:00<?, ?it/s]

9055
db1
db2
Your documents are roughly 244.4 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 2/2 [00:00<00:00, 16.86it/s]


Your documents are roughly 228.20000000000002 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


...

What is the date of Fall 2025 Registration Week；Spring 2025

November 18-November 22, Fall 2025Spring 2025 (No specific date mentioned in the context documents)
November 18-November 22, Fall 2025Spring 2025 (no specific date mentioned in the context)
Your documents are roughly 253.9 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 2/2 [00:00<00:00, 17.86it/s]


Your documents are roughly 281.90000000000003 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 2/2 [00:00<00:00, 16.71it/s]


What is the email of PhD Program Director in CMU LTI

callan@cs.cmu.edu
staceyy@cs.cmu.edu


In [53]:
# sys2, sys1, tie
ps_win = [0,0,0]
rs_win = [0,0,0]
f1_win = [0,0,0]

In [59]:
for ps1, ps2 in zip(ps_1, ps_2):
  if ps2 > ps1:
    ps_win[0] += 1
  elif ps2 < ps1:
    ps_win[1] += 1
  else:
    ps_win[2] += 1

for rs1, rs2 in zip(rs_1, rs_2):
  if rs2 > rs1:
    rs_win[0] += 1
  elif rs2 < rs1:
    rs_win[1] += 1
  else:
    rs_win[2] += 1

for f11, f12 in zip(f1_1, f1_2):
  if f12 > f11:
    f1_win[0] += 1
  elif f12 < f11:
    f1_win[1] += 1
  else:
    f1_win[2] += 1

print(ps_win)
print(rs_win)
print(f1_win)

[100, 0, 0]
[98, 2, 0]
[100, 0, 0]


In [60]:
# 50
ps_win = [x/float(sum(ps_win)) for x in ps_win]
rs_win = [x/float(sum(rs_win)) for x in rs_win]
f1_win = [x/float(sum(f1_win)) for x in f1_win]
print(ps_win)
print(rs_win)
print(f1_win)

if ps_win[0] > ps_win[1]:
  print('(sys2 is superior in ps with p value p=%.3f)\n' % (1 - ps_win[0]))
elif ps_win[1] > ps_win[0]:
  print('(sys1 is superior in ps with p value p=%.3f)\n' % (1 - ps_win[1]))

if rs_win[0] > rs_win[1]:
  print('(sys2 is superior in rs with p value p=%.3f)\n' % (1 - rs_win[0]))
elif rs_win[1] > rs_win[0]:
  print('(sys1 is superior in rs with p value p=%.3f)\n' % (1 - rs_win[1]))

if f1_win[0] > f1_win[1]:
  print('(sys2 is superior in f1 with p value p=%.3f)\n' % (1 - f1_win[0]))
elif f1_win[1] > f1_win[0]:
  print('(sys1 is superior in f1 with p value p=%.3f)\n' % (1 - f1_win[1]))

[1.0, 0.0, 0.0]
[0.98, 0.02, 0.0]
[1.0, 0.0, 0.0]
(sys2 is superior in ps with p value p=0.000)

(sys2 is superior in rs with p value p=0.020)

(sys2 is superior in f1 with p value p=0.000)

